In [1]:
from adodbapi.ado_consts import directions
from cartopy import crs as ccrs
import cartopy.feature as cfeature
import multiprocessing
import sys
import cartopy.feature as cfeature
import cmaps
import matplotlib.pyplot as plt
import numpy as np
import tqdm as tq
import xarray as xr
from cartopy.io.shapereader import Reader
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter  # 专门提供经纬度的
from cartopy.util import add_cyclic_point
from matplotlib import gridspec
from matplotlib import ticker
from matplotlib.pyplot import quiverkey
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.ndimage import filters

from toolbar.significance_test import corr_test
from toolbar.TN_WaveActivityFlux import TN_WAF_3D
from toolbar.curved_quivers.modplot import *
from toolbar.data_read import *


def corr(time_series, data):
    # 计算相关系数
    # 将 data 重塑为二维：时间轴为第一个维度
    reshaped_data = data.reshape(len(time_series), -1)

    # 减去均值以标准化
    time_series_mean = time_series - np.mean(time_series)
    data_mean = reshaped_data - np.mean(reshaped_data, axis=0)

    # 计算分子（协方差）
    numerator = np.sum(data_mean * time_series_mean[:, np.newaxis], axis=0)

    # 计算分母（标准差乘积）
    denominator = np.sqrt(np.sum(data_mean ** 2, axis=0)) * np.sqrt(np.sum(time_series_mean ** 2))

    # 相关系数
    correlation = numerator / denominator

    # 重塑为 (lat, lon)
    correlation_map = correlation.reshape(data.shape[1:])
    return correlation_map


K_type = xr.open_dataset(r"D:/PyFile/p2/data/Time_type_AverFiltAll0.9%_0.3%_3.nc")
# z
z_low = era5_p("E:/data/ERA5/ERA5_pressLev/era5_pressLev.nc", 1960, 2022, [200, 500, 850], 'z')
z_high = era5_hp("E:/data/ERA5/ERA5_pressLev/era5_pressLev_high.nc", 1960, 2022, [100, 150], 'z')
z = xr.concat([z_high, z_low], dim='level')
# u
u_low = era5_p("E:/data/ERA5/ERA5_pressLev/era5_pressLev.nc", 1960, 2022, [200, 500, 850], 'u')
u_high = era5_hp("E:/data/ERA5/ERA5_pressLev/era5_pressLev_high.nc", 1960, 2022, [100, 150], 'u')
u = xr.concat([u_high, u_low], dim='level')
# v
v_low = era5_p("E:/data/ERA5/ERA5_pressLev/era5_pressLev.nc", 1960, 2022, [200, 500, 850], 'v')
v_high = era5_hp("E:/data/ERA5/ERA5_pressLev/era5_pressLev_high.nc", 1960, 2022, [100, 150], 'v')
v = xr.concat([v_high, v_low], dim='level')
# t
t_low = era5_p("E:/data/ERA5/ERA5_pressLev/era5_pressLev.nc", 1960, 2022, [200, 500, 850], 't')
t_high = era5_hp("E:/data/ERA5/ERA5_pressLev/era5_pressLev_high.nc", 1960, 2022, [100, 150], 't')
t = xr.concat([t_high, t_low], dim='level')
# pre
pre = prec("E:/data/NOAA/PREC/precip.mon.anom.nc", 1960, 2022)
# sst
sst = ersst("E:/data/NOAA/ERSSTv5/sst.mnmean.nc", 1960, 2022)

In [ ]:
# 春季超前相关
Z = z.sel(time=slice('1961-01-01', '2022-12-31'))
Z = Z.sel(time=Z['time.month'].isin([3, 4, 5])).groupby('time.year').mean('time').transpose('year', 'level', 'lat',
                                                                                            'lon')
U = u.sel(time=slice('1961-01-01', '2022-12-31'))
U = U.sel(time=U['time.month'].isin([3, 4, 5])).groupby('time.year').mean('time').transpose('year', 'level', 'lat',
                                                                                            'lon')
V = v.sel(time=slice('1961-01-01', '2022-12-31'))
V = V.sel(time=V['time.month'].isin([3, 4, 5])).groupby('time.year').mean('time').transpose('year', 'level', 'lat',
                                                                                            'lon')
T = t.sel(time=slice('1961-01-01', '2022-12-31'))
T = T.sel(time=T['time.month'].isin([3, 4, 5])).groupby('time.year').mean('time').transpose('year', 'level', 'lat', 'lon')

Pre = pre.sel(time=slice('1961-01-01', '2022-12-31'))
Pre = Pre.sel(time=Pre['time.month'].isin([3, 4, 5])).groupby(
    'time.year').mean('time').transpose('year', 'lat', 'lon')
Sst = sst.sel(time=slice('1961-01-01', '2022-12-31'))
Sst = Sst.sel(time=Sst['time.month'].isin([3, 4, 5])).groupby(
    'time.year').mean('time').transpose('year', 'lat', 'lon')
corr_z = np.zeros((len(K_type['type']), len(Z['level']), len(Z['lat']), len(Z['lon'])))
reg_z = np.zeros((len(K_type['type']), len(Z['level']), len(Z['lat']), len(Z['lon'])))
corr_u = np.zeros((len(K_type['type']), len(U['level']), len(U['lat']), len(U['lon'])))
corr_v = np.zeros((len(K_type['type']), len(V['level']), len(V['lat']), len(V['lon'])))
corr_t = np.zeros((len(K_type['type']), len(T['level']), len(T['lat']), len(T['lon'])))
corr_pre = np.zeros((len(K_type['type']), len(Pre['lat']), len(Pre['lon'])))
corr_sst = np.zeros((len(K_type['type']), len(Sst['lat']), len(Sst['lon'])))

for i in tq.trange(len(K_type['type'])):
    time_series = K_type.sel(type=i + 1)['K'].data
    time_series = time_series - np.polyval(np.polyfit(range(len(time_series)), time_series, 1), range(len(time_series)))
    time_series = (time_series - np.mean(time_series)) / np.var(time_series)
    for j in tq.trange(len(z['level'])):
        lev = z['level'][j].data
        corr_z[i, j] = corr(time_series, Z['z'].sel(level=lev).data)
        corr_u[i, j] = corr(time_series, U['u'].sel(level=lev).data)
        corr_v[i, j] = corr(time_series, V['v'].sel(level=lev).data)
        corr_t[i, j] = corr(time_series, T['t'].sel(level=lev).data)
        reg_z[i, j] = np.array([np.polyfit(time_series, f, 1)[0] for f in
                                Z['z'].sel(level=lev).transpose('lat', 'lon', 'year').data.reshape(-1,
                                                                                                   len(time_series))]).reshape(
            Z['z'].sel(level=lev).data.shape[1], Z['z'].sel(level=lev).data.shape[2])
    corr_pre[i] = corr(time_series, Pre['pre'].data)
    corr_sst[i] = corr(time_series, Sst['sst'].data)

corr_nc = xr.Dataset(
    {
        'corr_z': (['type', 'level', 'lat', 'lon'], corr_z),
        'corr_u': (['type', 'level', 'lat', 'lon'], corr_u),
        'corr_v': (['type', 'level', 'lat', 'lon'], corr_v),
        'corr_t': (['type', 'level', 'lat', 'lon'], corr_t),
        'corr_pre': (['type', 'prelat', 'prelon'], corr_pre),
        'corr_sst': (['type', 'sstlat', 'sstlon'], corr_sst),
        'reg_z': (['type', 'level', 'lat', 'lon'], reg_z)
    },
    coords={
        'type': K_type['type'],
        'level': Z['level'],
        'lat': Z['lat'],
        'lon': Z['lon'],
        'prelat': Pre['lat'],
        'prelon': Pre['lon'],
        'sstlat': Sst['lat'],
        'sstlon': Sst['lon']
    }
)
corr_nc.to_netcdf(r"D:/PyFile/p2/data/corr_MAM.nc")


  0%|          | 0/5 [00:00<?, ?it/s]